In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import copy

import torch
import socialforce

_ = torch.manual_seed(43)

(pedped-1d-wall)=
# 1D Wall


## Parametric

This is an extension of the {ref}`pedped-1d` example to study the robustness
of the inference process to potentials with steep gradients.
We use a modified $V(b)$ potential that could be described as a "soft wall"
at $b=\sigma$. The amount of gradients is determined by the width $w$ of this 
wall:
\begin{align}
    V(b) &= \exp\left(- \frac{b - \sigma}{w} \right)
\end{align}
with its two parameters $\sigma$ and $w$.

In [ ]:
ped_ped = socialforce.potentials.PedPedPotentialWall(w=0.1)
with socialforce.show.canvas(ncols=2) as (ax1, ax2): 
    ax1.set_ylim(0.0, 3.0)
    ax2.set_ylim(-3.0, 0.0)
    socialforce.show.potential_1d_parametric(
        ped_ped, ax1, ax2, 
        label=r'true', sigma_label=r'true $\sigma$', color='gray')

## Scenario

We generate a single {ref}`Circle scenario <scenarios>`.

In [ ]:
circle = socialforce.scenarios.Circle(ped_ped=ped_ped)
scenario = circle.generate(1)
true_experience = socialforce.Trainer.scenes_to_experience(scenario)

with socialforce.show.track_canvas() as ax:
    socialforce.show.states(ax, scenario[0])

This synthetic path shows a non-smooth direction change of the orange 
pedestrian. This is an artifact stemming from the finite step size in
the simulation of the dynamics. We could remove this artifact by increasing
the oversampling in our simulation.

## MLP

We infer the parameters of an MLP to approximate the 1D scalar 
function $\textrm{SF}(b)$ above from synthetic observations.
The `PedPedPotentialMLP` is a two-layer MLP with softplus activations:
\begin{align}
    \textrm{MLP}(b) &= \textrm{Softplus} \;\; L_{1\times5} \;\; \textrm{Softplus} \;\; L_{5\times1} \;\; b
\end{align}
which is written in terms of linear and non-linear operators where
the Softplus operator applies the softplus function on its input from the right
and $L$ is a linear operator (a matrix) with the subscript indicating the 
$\textrm{output features} \times \textrm{input features}$.
This two-layer MLP with 5 hidden units has 10 parameters.


In [ ]:
V = socialforce.potentials.PedPedPotentialMLP(hidden_units=8)
initial_state_dict = copy.deepcopy(V.state_dict())

## Inference

We use a standard optimizer from PyTorch (SGD).
You can specify a standard PyTorch loss function for the `Trainer` as well
but here the default of a `torch.nn.L1Loss()` is used.

In [ ]:
simulator = socialforce.Simulator(ped_ped=V)
opt = torch.optim.SGD(V.parameters(), lr=3.0)
socialforce.Trainer(simulator, opt).loop(100, true_experience, log_interval=10)
final_state_dict = copy.deepcopy(V.state_dict())

In [ ]:
# HIDE CODE
with socialforce.show.canvas(ncols=2) as (ax1, ax2):
    ax1.set_ylim(0.0, 3.0)
    ax2.set_ylim(-3.0, 0.0)

    socialforce.show.potential_1d_parametric(
        circle.ped_ped, ax1, ax2, 
        label=r'true $V_0 e^{-b/\sigma}$', sigma_label=r'true $\sigma$', color='gray')

    V.load_state_dict(initial_state_dict)
    socialforce.show.potential_1d(V, ax1, ax2, label=r'initial MLP($b$)', linestyle='dashed', color='C0')

    V.load_state_dict(final_state_dict)
    socialforce.show.potential_1d(V, ax1, ax2, label=r'MLP($b$)', color='C0')

We can see that the numerical challenges impact our ability to infer the 
parameters of this potential. When we reduce $w$ further, this discrepancy grows.